<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

* 基于  kipf_gcn_keras: https://github.com/tkipf/keras-gcn  + William_GraphSAGE_tf : https://github.com/williamleif/GraphSAGE

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn.manifold import TSNE

import os
import pickle as pkl
import sys

import tensorflow as tf
from tensorflow.python.keras.initializers import glorot_uniform, Zeros
from tensorflow.python.keras.layers import Input, Dense, Dropout, Layer, LSTM
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.layers import Lambda

In [12]:
# 所需函数
def get_splits(y, ):
    
    idx_list = np.arange(len(y))
    
    # 训练集中每个类别均分布 各 20个。
    idx_train = []
    label_count = {}
    for i, label in enumerate(y):
        label = np.argmax(label)
        if label_count.get(label, 0) < 20:
            idx_train.append(i)
            label_count[label] = label_count.get(label, 0) + 1
    
    #  train： 140；  val： 500；  test：1000
    idx_val_test = list(set(idx_list) - set(idx_train))
    idx_val = idx_val_test[0:500]
    idx_test = idx_val_test[500:1500]
    
    y_train = np.zeros(y.shape, dtype=np.int32)
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train]
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]
    train_mask = sample_mask(idx_train, y.shape[0])
    val_mask = sample_mask(idx_val, y.shape[0])
    test_mask = sample_mask(idx_test, y.shape[0])

    return y_train, y_val, y_test, train_mask, val_mask, test_mask


def get_splits_kipf(y, ):
    
    
    idx_train = range(140) 
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    
    y_train = np.zeros(y.shape, dtype=np.int32)
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train]
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]
    train_mask = sample_mask(idx_train, y.shape[0])
    val_mask = sample_mask(idx_val, y.shape[0])
    test_mask = sample_mask(idx_test, y.shape[0])

    return y_train, y_val, y_test, train_mask, val_mask, test_mask

def get_splits_0(y):
    """
    数据集划分：
    
    y : 标签数据 即 load_data() 函数 返回的 labels 数据 
    数据格式  (2708 , 7 ) 2708 个数据，一共 7 个 class ， one-hot 维度 7 维 
    """

    idx_train = range(140)  # ??? 少一部分数据啊
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    y_train = np.zeros(y.shape, dtype=np.int32) # 初始化 (2708,7) size 的全 0 矩阵。
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train] #  取 y 矩阵前 140 行， 其余行赋值为 0 
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]

    # 训练数据的样本掩码
    train_mask = sample_mask(idx_train, y.shape[0]) # 前 140行为 true， 后面为 false 的向量
    return y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask


def load_data_0(path="data/cora/", dataset="cora"):
    """
    Load citation network dataset 
    
    (cora only for now)
    """

    print('Loading {} dataset...'.format(dataset))

    ### 读取样本id，特征和标签
    idx_features_labels = np.genfromtxt(
        "{}{}.content".format(path, dataset),
        dtype=np.dtype(str))  # np.genfromtxt()生成 array
    features = sp.csr_matrix(idx_features_labels[:, 1:-1],
                             dtype=np.float32)  # 提取样本的特征，并将其转换为csr矩阵
    labels = encode_onehot(
        idx_features_labels[:, -1])  # 提取样本的标签，并将其转换为one-hot编码形式
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)  # 样本的id数组
    idx_map = {j: i for i, j in enumerate(idx)}  # 创建一个字典储存数据id

    ### 读取样本之间关系 ： 连边
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(
                         edges_unordered.shape)  # 无序边  map 成为有序
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)  # 构建图的邻接矩阵
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(
        adj.T > adj)  # 将非对称邻接矩阵转变为对称邻接矩阵

    print('Dataset has {} nodes, {} edges, {} features.'.format(
        adj.shape[0], edges.shape[0], features.shape[1]))

    return features.todense(), adj, labels  # 返回特征的密集矩阵表示、邻接矩阵和标签的one-hot编码



def load_data_v1(dataset="cora",path="data/cora/"):
    
    
    # 提取数据
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    onehot_labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(onehot_labels.shape[0], onehot_labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    # adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj = convert_symmetric(adj, )

    print('Dataset has {} nodes, {} edges, {} features.'.format(
        adj.shape[0], edges.shape[0], features.shape[1]))

    y_train, y_val, y_test, train_mask, val_mask, test_mask = get_splits_kipf(
        onehot_labels)  #  分割标签

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask


def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def load_data(dataset_str):
    """Load data."""
    FILE_PATH = os.path.abspath(__file__)
    DIR_PATH = os.path.dirname(FILE_PATH)
    DATA_PATH = os.path.join(DIR_PATH, 'data/')
    DATA_PATH = "../data/cora/"

    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("{}ind.{}.{}".format(DATA_PATH, dataset_str, names[i]),
                  'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("{}ind.{}.test.index".format(
        DATA_PATH, dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder),
                                    max(test_idx_reorder) + 1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range - min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range - min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y) + 500)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    return sp.csr_matrix(
        adj), features, y_train, y_val, y_test, train_mask, val_mask, test_mask


def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def convert_symmetric(X, sparse=True):
    if sparse:
        X += X.T - sp.diags(X.diagonal())
    else:
        X += X.T - np.diag(X.diagonal())
    return X


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {
        c: np.identity(len(classes))[i, :]
        for i, c in enumerate(classes)
    }
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def normalize_adj(adj, symmetric=True):
    if symmetric:
        d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
        a_norm = adj.dot(d).transpose().dot(d).tocsr()
    else:
        d = sp.diags(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj).tocsr()
    return a_norm


def preprocess_adj(adj, symmetric=True):
    adj = adj + sp.eye(adj.shape[0])
    adj = normalize_adj(adj, symmetric)
    return adj


def plot_embeddings(embeddings, X, Y):

    emb_list = []
    for k in X:
        emb_list.append(embeddings[k])
    emb_list = np.array(emb_list)

    model = TSNE(n_components=2)
    node_pos = model.fit_transform(emb_list)

    color_idx = {}
    for i in range(len(X)):
        color_idx.setdefault(Y[i][:], [])
        color_idx[Y[i][:]].append(i)

    for c, idx in color_idx.items():
        plt.scatter(node_pos[idx, 0], node_pos[idx, 1], label=c)
    plt.legend()
    plt.show()


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features.todense()

def categorical_crossentropy(preds, labels):
    """
    损失函数计算： 所有有标签的数据，主要用于训练时进行梯度下降 ： 
    
    L=Y*ln(Z)
    
    param preds: 模型对样本的输出数组
    param labels: 样本的one-hot标签数组
    return: 样本的平均交叉熵损失
    """
    return np.mean(-np.log(np.extract(
        labels, preds)))  # np.extract(condition, x)函数，根据某个条件从数组中抽取元素


def accuracy(preds, labels):
    """
    计算准确率
    """
    # np.argmax(x,1) 每一行元素最大值索引  : 相当于将 one-hot 转成一个数值 [0,1,0,0,0] = 1
    # np.equal(x1, x2) 比较 array 对应元素是否相等 返回 np.array([ True,  True, False])
    # np.mean(np.array([ True,  True, False])) = 0.666
    return np.mean(np.equal(np.argmax(labels, 1), np.argmax(preds, 1)))


def evaluate_preds(preds, labels, indices):
    """
     评估样本划分的损失函数和准确率
     
    :param preds:对于样本的预测值
    :param labels:样本的标签one-hot向量
    :param indices:样本的索引集合
    :return:交叉熵损失函数列表、准确率列表
    """
    split_loss = list()
    split_acc = list()

    for y_split, idx_split in zip(labels, indices):
        # 计算每一个样本划分的交叉熵损失函数
        split_loss.append(
            categorical_crossentropy(preds[idx_split], y_split[idx_split]))
        # 计算每一个样本划分的准确率
        split_acc.append(accuracy(preds[idx_split], y_split[idx_split]))

    return split_loss, split_acc


In [7]:
# GraphSAGE 模型
class MeanAggregator(Layer):

    def __init__(self, units, input_dim, neigh_max, concat=True, dropout_rate=0.0, activation=tf.nn.relu, l2_reg=0,
                 use_bias=False,
                 seed=1024, **kwargs):
        super(MeanAggregator, self).__init__()
        self.units = units
        self.neigh_max = neigh_max
        self.concat = concat
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        self.use_bias = use_bias
        self.activation = activation
        self.seed = seed
        self.input_dim = input_dim

    def build(self, input_shapes):

        self.neigh_weights = self.add_weight(shape=(self.input_dim, self.units),
                                             initializer=glorot_uniform(
                                                 seed=self.seed),
                                             regularizer=l2(self.l2_reg),
                                             name="neigh_weights")
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units), initializer=Zeros(),
                                        name='bias_weight')

        self.dropout = Dropout(self.dropout_rate)
        self.built = True

    def call(self, inputs, training=None):
        features, node, neighbours = inputs

        node_feat = tf.nn.embedding_lookup(features, node)
        neigh_feat = tf.nn.embedding_lookup(features, neighbours)

        node_feat = self.dropout(node_feat, training=training)
        neigh_feat = self.dropout(neigh_feat, training=training)

        concat_feat = tf.concat([neigh_feat, node_feat], axis=1)
        concat_mean = tf.reduce_mean(concat_feat, axis=1, keep_dims=False)

        output = tf.matmul(concat_mean, self.neigh_weights)
        if self.use_bias:
            output += self.bias
        if self.activation:
            output = self.activation(output)

        # output = tf.nn.l2_normalize(output,dim=-1)
        output._uses_learning_phase = True

        return output

    def get_config(self):
        config = {'units': self.units,
                  'concat': self.concat,
                  'seed': self.seed
                  }

        base_config = super(MeanAggregator, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class PoolingAggregator(Layer):

    def __init__(self, units, input_dim, neigh_max, aggregator='meanpooling', concat=True,
                 dropout_rate=0.0,
                 activation=tf.nn.relu, l2_reg=0, use_bias=False,
                 seed=1024, ):
        super(PoolingAggregator, self).__init__()
        self.output_dim = units
        self.input_dim = input_dim
        self.concat = concat
        self.pooling = aggregator
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        self.use_bias = use_bias
        self.activation = activation
        self.neigh_max = neigh_max
        self.seed = seed

        # if neigh_input_dim is None:

    def build(self, input_shapes):

        self.dense_layers = [Dense(
            self.input_dim, activation='relu', use_bias=True, kernel_regularizer=l2(self.l2_reg))]

        self.neigh_weights = self.add_weight(
            shape=(self.input_dim * 2, self.output_dim),
            initializer=glorot_uniform(
                seed=self.seed),
            regularizer=l2(self.l2_reg),

            name="neigh_weights")

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.output_dim,),
                                        initializer=Zeros(),
                                        name='bias_weight')

        self.built = True

    def call(self, inputs, mask=None):

        features, node, neighbours = inputs

        node_feat = tf.nn.embedding_lookup(features, node)
        neigh_feat = tf.nn.embedding_lookup(features, neighbours)

        dims = tf.shape(neigh_feat)
        batch_size = dims[0]
        num_neighbors = dims[1]
        h_reshaped = tf.reshape(
            neigh_feat, (batch_size * num_neighbors, self.input_dim))

        for l in self.dense_layers:
            h_reshaped = l(h_reshaped)
        neigh_feat = tf.reshape(
            h_reshaped, (batch_size, num_neighbors, int(h_reshaped.shape[-1])))

        if self.pooling == "meanpooling":
            neigh_feat = tf.reduce_mean(neigh_feat, axis=1, keep_dims=False)
        else:
            neigh_feat = tf.reduce_max(neigh_feat, axis=1)

        output = tf.concat(
            [tf.squeeze(node_feat, axis=1), neigh_feat], axis=-1)

        output = tf.matmul(output, self.neigh_weights)
        if self.use_bias:
            output += self.bias
        if self.activation:
            output = self.activation(output)

        # output = tf.nn.l2_normalize(output, dim=-1)

        return output

    def get_config(self):
        config = {'output_dim': self.output_dim,
                  'concat': self.concat
                  }

        base_config = super(PoolingAggregator, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def GraphSAGE(feature_dim, neighbor_num, n_hidden, n_classes, use_bias=True, activation=tf.nn.relu,
              aggregator_type='mean', dropout_rate=0.0, l2_reg=0):
    
    """
    feature_dim : 输入特征维度
    neighbor_num : 
    n_hidden : 
    
    
    
    """
    features = Input(shape=(feature_dim,))
    node_input = Input(shape=(1,), dtype=tf.int32)
    neighbor_input = [Input(shape=(l,), dtype=tf.int32) for l in neighbor_num]

    if aggregator_type == 'mean':
        aggregator = MeanAggregator
    else:
        aggregator = PoolingAggregator

    h = features
    for i in range(0, len(neighbor_num)):
        if i > 0:
            feature_dim = n_hidden
        if i == len(neighbor_num) - 1:
            activation = tf.nn.softmax
            n_hidden = n_classes
        h = aggregator(units=n_hidden, input_dim=feature_dim, activation=activation, l2_reg=l2_reg, use_bias=use_bias,
                       dropout_rate=dropout_rate, neigh_max=neighbor_num[i], aggregator=aggregator_type)(
            [h, node_input, neighbor_input[i]])  #

    output = h
    input_list = [features, node_input] + neighbor_input
    model = Model(input_list, outputs=output)
    return model

In [8]:
def sample_neighs(G, nodes, sample_num=None, self_loop=False, shuffle=True):  # 抽样邻居节点
    _sample = np.random.choice
    neighs = [list(G[int(node)]) for node in nodes]  # nodes里每个节点的邻居
    if sample_num:
        if self_loop:
            sample_num -= 1

        samp_neighs = [
            list(_sample(neigh, sample_num, replace=False)) if len(neigh) >= sample_num else list(
                _sample(neigh, sample_num, replace=True)) for neigh in neighs]  # 采样邻居 : 如果大于采样数，就无放回采样
        if self_loop:
            samp_neighs = [
                samp_neigh + list([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]  # gcn邻居要加上自己

        if shuffle:
            samp_neighs = [list(np.random.permutation(x)) for x in samp_neighs] #  对采样得到的邻居随机排序
    else:
        samp_neighs = neighs
    return np.asarray(samp_neighs), np.asarray(list(map(len, samp_neighs)))

In [14]:
DATASET = 'cora'
features, A, y = load_data_0(
    dataset=DATASET)  # X: 特征（2708，2708）、 A: 邻接矩阵 （2708，2708）、 y: 标签 （2708，7）
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits_0(
    y)

features /= features.sum(axis=1, ).reshape(-1, 1)
G = nx.from_scipy_sparse_matrix(A, create_using=nx.DiGraph())

A = preprocess_adj(A)

indexs = np.arange(A.shape[0])
neigh_number = [10, 25]
neigh_maxlen = []

model_input = [features,
               np.asarray(indexs, dtype=np.int32)]  # np.asarray 将列表转换为一个 array

for num in neigh_number:
    sample_neigh, sample_neigh_len = sample_neighs(G,
                                                   indexs,
                                                   num,
                                                   self_loop=False)
    model_input.extend([sample_neigh])
    neigh_maxlen.append(max(sample_neigh_len))
    

model = GraphSAGE(feature_dim=features.shape[1],
                  neighbor_num=neigh_maxlen,
                  n_hidden=16,
                  n_classes=y_train.shape[1],
                  use_bias=True,
                  activation=tf.nn.relu,
                  aggregator_type='mean',
                  dropout_rate=0.5,
                  l2_reg=2.5e-4)
model.compile(
    Adam(0.01),
    'categorical_crossentropy',
)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [17]:
NB_EPOCH = 200
#val_data = (model_input, y_val, val_mask)


for epoch in range(1, NB_EPOCH + 1):
    model.fit(model_input,
          y_train,
          sample_weight=train_mask,
          batch_size=A.shape[0],
          epochs=1,
          shuffle=False,
          verbose=0)
    # Predict on full dataset
    
    preds = model.predict(model_input, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          )
    
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:", "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))

Epoch: 0001 train_loss= 1.9041 train_acc= 0.2214 val_loss= 1.9110 val_acc= 0.1467
Epoch: 0002 train_loss= 1.8921 train_acc= 0.3929 val_loss= 1.9010 val_acc= 0.3000
Epoch: 0003 train_loss= 1.8801 train_acc= 0.5286 val_loss= 1.8907 val_acc= 0.4867
Epoch: 0004 train_loss= 1.8676 train_acc= 0.4786 val_loss= 1.8799 val_acc= 0.4333
Epoch: 0005 train_loss= 1.8547 train_acc= 0.4214 val_loss= 1.8688 val_acc= 0.3700
Epoch: 0006 train_loss= 1.8416 train_acc= 0.4071 val_loss= 1.8578 val_acc= 0.3700
Epoch: 0007 train_loss= 1.8290 train_acc= 0.4000 val_loss= 1.8472 val_acc= 0.3633
Epoch: 0008 train_loss= 1.8162 train_acc= 0.3929 val_loss= 1.8365 val_acc= 0.3633
Epoch: 0009 train_loss= 1.8034 train_acc= 0.3929 val_loss= 1.8256 val_acc= 0.3633
Epoch: 0010 train_loss= 1.7907 train_acc= 0.3929 val_loss= 1.8146 val_acc= 0.3633
Epoch: 0011 train_loss= 1.7782 train_acc= 0.3929 val_loss= 1.8038 val_acc= 0.3633
Epoch: 0012 train_loss= 1.7662 train_acc= 0.3929 val_loss= 1.7933 val_acc= 0.3633
Epoch: 0013 trai

Epoch: 0101 train_loss= 0.7417 train_acc= 0.8857 val_loss= 1.0118 val_acc= 0.7767
Epoch: 0102 train_loss= 0.7353 train_acc= 0.8857 val_loss= 1.0067 val_acc= 0.7767
Epoch: 0103 train_loss= 0.7289 train_acc= 0.8857 val_loss= 1.0017 val_acc= 0.7800
Epoch: 0104 train_loss= 0.7228 train_acc= 0.8929 val_loss= 0.9964 val_acc= 0.7800
Epoch: 0105 train_loss= 0.7167 train_acc= 0.8929 val_loss= 0.9922 val_acc= 0.7767
Epoch: 0106 train_loss= 0.7106 train_acc= 0.9071 val_loss= 0.9890 val_acc= 0.7733
Epoch: 0107 train_loss= 0.7048 train_acc= 0.9071 val_loss= 0.9861 val_acc= 0.7733
Epoch: 0108 train_loss= 0.6990 train_acc= 0.9071 val_loss= 0.9827 val_acc= 0.7733
Epoch: 0109 train_loss= 0.6935 train_acc= 0.9000 val_loss= 0.9783 val_acc= 0.7733
Epoch: 0110 train_loss= 0.6880 train_acc= 0.9000 val_loss= 0.9726 val_acc= 0.7767
Epoch: 0111 train_loss= 0.6827 train_acc= 0.9000 val_loss= 0.9661 val_acc= 0.7833
Epoch: 0112 train_loss= 0.6776 train_acc= 0.9000 val_loss= 0.9603 val_acc= 0.7800
Epoch: 0113 trai